In [416]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [417]:
file = r'/Users/richard/Documents/WSC - variable cross-check_v1.xlsx'
data = r'/Users/richard/Downloads/wsc-dataset-0.2.0.csv'

In [418]:
# Had to add the nrows variable as the Excel was pulling in 1,000s of rows for some reason
df = pd.read_excel(file)
data_df = pd.read_csv(data)

In [419]:
df.tail()

,Unnamed: 0,Check,Patient input,Dr Input,Calculated,Feature,Target,Proposed Removal,Action to take,Composite,Unnamed: 10
234,se,NaN,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,NaN
235,waso,NaN,NaN,D,NaN,NaN,T,NaN,NaN,NaN,NaN
236,mean_desat_perc,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN
237,mean_desat_dur,NaN,NaN,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN
238,cvd_stroke_death,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [420]:
data_df.head()

,wsc_id,wsc_vst,vst_year,sex,age,race,education_survey1,creatinine,glucose,hdl,...,pcttstrem,avgo2sattst,minsao2tst,ptstl90,sleep_latency,rem_latency,se,waso,mean_desat_perc,mean_desat_dur
0,10119,1,2003,M,52,5,6.0,0.9,89.0,52.0,...,10.5,96.7,93.8,0.0,7.0,87.0,83.0,65.0,3.42,27.4
1,10119,2,2008,M,58,5,6.0,0.8,95.0,36.0,...,10.8,96.9,86.4,0.0,4.0,145.5,93.3,27.0,3.65,31.0
2,10191,1,2001,F,50,5,5.0,0.8,96.0,77.0,...,30.8,97.2,94.2,0.0,7.0,117.0,87.1,44.0,3.70,29.1
3,10191,2,2004,F,53,5,5.0,0.7,74.0,104.0,...,22.5,96.6,85.9,0.0,5.0,95.0,85.4,62.5,3.62,37.9
4,10198,1,2003,F,57,5,4.0,0.9,103.0,48.0,...,18.1,95.3,87.2,0.7,7.0,109.5,83.7,58.0,4.12,37.0


In [421]:
deleted = df[df['Proposed Removal'] == 'R']
deleted_cols = deleted.iloc[:, 0]

In [422]:
deleted_cols.to_list()

['vst_year',
 'arm_left1',
 'arm_left2',
 'arm_right1',
 'arm_right2',
 'ankle_left1',
 'ankle_left2',
 'ankle_right1',
 'ankle_right2',
 'sit_sys_repeat',
 'sit_dia_repeat',
 'arm_left_repeat',
 'arm_right_repeat',
 'ankle_left_repeat',
 'ankle_right_repeat',
 'sit_sys_repeat2',
 'sit_dia_repeat2',
 'arm_left_repeat2',
 'arm_right_repeat2',
 'ankle_left_repeat2',
 'ankle_right_repeat2',
 'ep1',
 'ep2',
 'ep3',
 'ep4',
 'ep5',
 'ep6',
 'ep7',
 'ep8',
 'zung1_scored',
 'zung2_scored',
 'zung3_scored',
 'zung4_scored',
 'zung5_scored',
 'zung6_scored',
 'zung7_scored',
 'zung8_scored',
 'zung9_scored',
 'zung10_scored',
 'zung11_scored',
 'zung12_scored',
 'zung13_scored',
 'zung14_scored',
 'zung15_scored',
 'zung16_scored',
 'zung17_scored',
 'zung18_scored',
 'zung19_scored',
 'zung20_scored',
 'smoker',
 'totsleepnap',
 'ps_diff',
 'ps_backsleep',
 'ps_wakerepeat',
 'ps_tooearly',
 'ps_notrested',
 'ps_wakeup',
 'ps_nightmare',
 'ninsomnia',
 'ninsomniadays',
 'nasal_cong_today',
 'n

In [423]:
data_df = data_df.drop(deleted_cols.to_list(), axis=1)

In [424]:
data_df.isnull().sum().sort_values(ascending=False)/len(data_df)

psg_oxygen         0.997665
psg_cpap           0.935019
cigars_day         0.919844
bowls_day          0.917899
thyroid_problem    0.867704
                     ...   
coronary_ynd       0.000000
awake_freq         0.000000
apnea_freq         0.000000
choke_freq         0.000000
waso               0.000000
Length: 134, dtype: float64

In [425]:
balance_cutoff = 0.9
imbalanced_classes = []
for col in data_df.columns:
    _ = data_df.columns.get_loc(col)
    if data_df.iloc[:, _].value_counts(normalize=True).head(1).values > balance_cutoff:
        imbalanced_classes.append((col, data_df.iloc[:, _].value_counts(normalize=True).head(1).values.astype(float)))

In [426]:
imbalanced_classes

[('race', array([0.95836576])),
 ('nasal_cong_none', array([1.])),
 ('coronary_ynd', array([0.9233463])),
 ('angina_ynd', array([0.96730245])),
 ('atheroscl_ynd', array([0.9758661])),
 ('heartattack_ynd', array([0.95679253])),
 ('congestivehf_ynd', array([0.98871595])),
 ('coronarybypass_ynd', array([0.95525292])),
 ('stroke_ynd', array([0.97743191])),
 ('emphysema_ynd', array([0.97898833])),
 ('angioplasty_ynd', array([0.95758755])),
 ('pacemaker_ynd', array([0.99105058])),
 ('coronary_artery_stent_ynd', array([0.9766537])),
 ('asthma_med', array([0.92140078])),
 ('asthma_rescue_med', array([0.96031128])),
 ('asthma_control_med', array([0.93696498])),
 ('dep_maoi_med', array([0.99922179])),
 ('dep_tca_med', array([0.9766537])),
 ('htn_alpha_med', array([0.96264591])),
 ('htn_arb_med', array([0.93190661])),
 ('narcotics_med', array([0.9766537])),
 ('decongestants_med', array([0.95914397])),
 ('anxiety_med', array([0.92801556])),
 ('estrogen_med', array([0.93385214])),
 ('androgen_med',

In [427]:
len(imbalanced_classes)

30

In [428]:
imbalanced_list = []
for classes in imbalanced_classes:
    imbalanced_list.append(classes[0])

In [429]:
imbalanced_list

['race',
 'nasal_cong_none',
 'coronary_ynd',
 'angina_ynd',
 'atheroscl_ynd',
 'heartattack_ynd',
 'congestivehf_ynd',
 'coronarybypass_ynd',
 'stroke_ynd',
 'emphysema_ynd',
 'angioplasty_ynd',
 'pacemaker_ynd',
 'coronary_artery_stent_ynd',
 'asthma_med',
 'asthma_rescue_med',
 'asthma_control_med',
 'dep_maoi_med',
 'dep_tca_med',
 'htn_alpha_med',
 'htn_arb_med',
 'narcotics_med',
 'decongestants_med',
 'anxiety_med',
 'estrogen_med',
 'androgen_med',
 'progesterone_med',
 'sedative_med',
 'stimulants_med',
 'psg_cpap',
 'psg_oxygen']

In [430]:
data_df.drop(imbalanced_list, axis=1, inplace=True)
data_df.drop_duplicates('wsc_id', inplace=True)
data_df.set_index('wsc_id', inplace=True)
data_df.fillna(0, inplace=True)

In [431]:
data_df

,wsc_vst,sex,age,education_survey1,creatinine,glucose,hdl,ldl,total_cholesterol,triglycerides,...,diabetes_med,thyroid_med,tst,tst_rem,tst_nrem,ahi,minsao2tst,ptstl90,se,waso
wsc_id,,,,,,,,,,,,,,,,,,,,,
10119,1,M,52,6.0,0.90,89.0,52.0,149.0,228.0,137.0,...,0,0,368.0,38.5,329.5,0.5,93.8,0.0,83.0,65.0
10191,1,F,50,5.0,0.80,96.0,77.0,122.0,222.0,117.0,...,0,0,419.0,129.0,290.0,0.0,94.2,0.0,87.1,44.0
10198,1,F,57,4.0,0.90,103.0,48.0,146.0,211.0,85.0,...,0,1,384.0,69.5,314.5,7.3,87.2,0.7,83.7,58.0
10226,1,M,55,4.0,1.40,131.0,39.0,107.0,173.0,135.0,...,0,0,328.0,46.5,281.5,18.1,89.8,0.0,71.4,91.0
10241,1,F,51,4.0,0.90,102.0,37.0,135.0,228.0,281.0,...,0,0,350.5,47.0,303.5,18.3,84.8,2.0,81.6,66.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99481,1,M,60,5.0,1.12,99.0,57.0,116.0,193.0,99.0,...,0,0,380.0,67.5,312.5,4.1,0.0,0.4,78.0,95.7
99651,1,M,69,5.0,1.40,206.0,36.0,112.0,184.0,181.0,...,0,0,374.0,36.0,338.0,10.3,76.9,2.5,87.5,47.0
99761,1,F,71,3.0,0.80,122.0,73.0,125.0,234.0,178.0,...,0,0,340.5,54.0,286.5,9.7,89.8,0.0,84.6,44.5
